<a href="https://colab.research.google.com/github/paarthpatel10/model1/blob/main/Model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
# importing dataset
df = pd.read_csv('/content/crop_yield.csv')

In [ ]:
#filtering data whose area is less than 1000 hectares
new_df= df[df['Area']>=1000]
len(new_df)
new_df= new_df[new_df['Production']>=1000]
len(new_df)
new_df= new_df[new_df['Pesticide']>=1000]
len(new_df)
new_df= new_df[new_df['Fertilizer']>=1000]
len(new_df)

12234

In [ ]:
X = new_df.drop('Yield', axis=1)
y = new_df["Yield"]
#Splitting off the test set (10% of the total data)
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

#Splitting the remaining data into training set (80% of total) and validation set (10% of total)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.1, random_state=42)
# 80% * 0.1111 = 10% (validation)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np

# Categorical and numerical features
categorical_features =['Crop', 'Season', 'State', 'Crop_Year']
numerical_features = ["Area", "Production", "Annual_Rainfall", "Fertilizer", "Pesticide"]

# Pipeline for preprocessing (OneHot for categorical and scaling for numerical)
one_hot = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
std_sca = StandardScaler()

# Preprocessing pipeline
transformer = ColumnTransformer([("One_hot", one_hot, categorical_features),
                                 ("std_sca", std_sca, numerical_features)],
                                remainder="passthrough")

In [ ]:
import warnings
warnings.filterwarnings('ignore')  # Ignore all warnings

from sklearn.experimental import enable_halving_search_cv  # Enable HalvingRandomSearchCV
from sklearn.model_selection import HalvingRandomSearchCV, RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error
from sklearn.model_selection import cross_val_score
import numpy as np

# Define your transformer here if it's not already defined
# transformer = ...

# Define the pipeline with a placeholder regressor
combined_pipeline = Pipeline(steps=[
    ("preprocessor", transformer),  # Your preprocessing step
    ("regressor", LinearRegression())  # Placeholder, will be replaced during search
])

# Hyperparameter grids with a few important parameters
# 1. RandomForest with RandomizedSearchCV


# Ridge
ridge_param_dist = {
    "regressor": [Ridge()],
    "regressor__alpha": np.logspace(-2, 1, 3),  # Fewer alpha values
}
knn_param_grid = {
   "regressor": [KNeighborsRegressor()],
    "regressor__n_neighbors": [1,3, 5, 7],  # Fewer values for neighbors
    "regressor__weights": ['uniform', 'distance'],  # Try uniform first
    "regressor__p": [1,2] ,
}
rf_param_dist = {
    "regressor": [RandomForestRegressor()],
    "regressor__n_estimators": [100, 200],
    "regressor__max_depth": [10, 20]
}

# LinearRegression does not have hyperparameters, so no param grid is needed

# Search configurations

linear_search = GridSearchCV(combined_pipeline, param_grid={}, cv=3)
ridge_search = HalvingRandomSearchCV(combined_pipeline, param_distributions=ridge_param_dist, n_candidates=50, cv=3, random_state=42)
knn_search = GridSearchCV(combined_pipeline, param_grid=knn_param_grid, cv=3)
rf_search = RandomizedSearchCV(combined_pipeline, param_distributions=rf_param_dist, n_iter=10, cv=3, random_state=42)

# Fit the models


linear_search.fit(X_train, y_train)


ridge_search.fit(X_train, y_train)
print("Ridge done")

knn_search.fit(X_train, y_train)
print("KNN done")

rf_search.fit(X_train, y_train)
print("RF done")


# Output the best models and their parameters
best_models = [

    {"model": linear_search.best_estimator_, "parameters": linear_search.best_params_},
    {"model": ridge_search.best_estimator_, "parameters": ridge_search.best_params_},
    {"model": knn_search.best_estimator_, "parameters": knn_search.best_params_},
    {"model": rf_search.best_estimator_, "parameters": rf_search.best_params_},
]

from tabulate import tabulate
import pandas as pd

# Initialize an empty DataFrame to store the evaluation metrics
metrics_df = pd.DataFrame(columns=["Model", "MSE_Train", "MSE_Val", "MSE_Test","MAE_Train", "MAE_Val","MAE Test", "R2_Train", "R2_Val","R2_Test","RMSE_Train","RMSE_Val","RMSE_Test" ])

# Define the evaluation function and store the results in a DataFrame
def evaluate_model(model, X_train, y_train, X_val, y_val,X_test, y_test, model_name):
    # Predicting the target values using the model and validation features
    y_predt = model.predict(X_train)
    y_pred = model.predict(X_val)
    y_predte = model.predict(X_test)

    # Calculating errors and metrics
    mset = mean_squared_error(y_train, y_predt)
    mse = mean_squared_error(y_val, y_pred)
    msete = mean_squared_error(y_test, y_predte)

    maet = mean_absolute_error(y_train, y_predt)
    mae = mean_absolute_error(y_val, y_pred)
    maete = mean_absolute_error(y_test, y_predte)

    r2t = r2_score(y_train, y_predt)
    r2 = r2_score(y_val, y_pred)
    r2te = r2_score(y_test, y_predte)

    rmset = np.sqrt(mset)
    rmse = np.sqrt(mse)
    rmsete = np.sqrt(msete)

    # Append the metrics to the DataFrame
    metrics_df.loc[len(metrics_df)] = [
        model_name,   # Model name
        mset,         # MSE on training data
        mse,          # MSE on validation data
        msete,
        maet,         # MAE on training data
        mae,
        maete,    # MAE on validation data
        r2t,          # R2 on training data
        r2,           # R2 on validation data
        r2te,
        rmset,        # RMSE on training data
        rmse,         # RMSE on validation data
        rmsete

    ]

# Evaluate all models
for result in best_models:
    model = result["model"]
    params = result["parameters"]
    model_name = type(model.named_steps['regressor']).__name__ if 'regressor' in model.named_steps else type(model).__name__

    # Evaluate the model and store the results
    evaluate_model(model, X_train, y_train, X_val, y_val,X_test, y_test, model_name)

# Use tabulate to print the metrics DataFrame in a tabular format
print(tabulate(metrics_df, headers='keys', tablefmt='grid'))



Linear done
Ridge done
KNN done
RF done
+----+-----------------------+-------------+-----------+------------+-------------+-----------+------------+------------+----------+-----------+--------------+------------+-------------+
|    | Model                 |   MSE_Train |   MSE_Val |   MSE_Test |   MAE_Train |   MAE_Val |   MAE Test |   R2_Train |   R2_Val |   R2_Test |   RMSE_Train |   RMSE_Val |   RMSE_Test |
+====+=======================+=============+===========+============+=============+===========+============+============+==========+===========+==============+============+=============+
|  0 | LinearRegression      |   107684    |  140927   |  108779    |    71.8811  |   81.324  |    71.2697 |   0.879097 | 0.907768 |  0.900455 |     328.152  |    375.403 |    329.816  |
+----+-----------------------+-------------+-----------+------------+-------------+-----------+------------+------------+----------+-----------+--------------+------------+-------------+
|  1 | Ridge             

In [ ]:
combined_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('One_hot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['Crop', 'Season', 'State',
                                                   'Crop_Year']),
                                                 ('std_sca', StandardScaler(),
                                                  ['Area', 'Production',
                                                   'Annual_Rainfall',
                                                   'Fertilizer',
                                                   'Pesticide'])])),
                ('regressor', LinearRegression())])

In [ ]:
# Dictionary to hold individual DataFrames for each model
model_metrics_tables = {}

# Define the evaluation function and store the results in separate DataFrames
def evaluate_model(model, X_train, y_train, X_val, y_val, X_test, y_test, model_name):
    # Predicting the target values using the model and validation features
    y_predt = model.predict(X_train)
    y_pred = model.predict(X_val)
    y_predte = model.predict(X_test)

    # Calculating errors and metrics
    mset = mean_squared_error(y_train, y_predt)
    mse = mean_squared_error(y_val, y_pred)
    msete = mean_squared_error(y_test, y_predte)

    maet = mean_absolute_error(y_train, y_predt)
    mae = mean_absolute_error(y_val, y_pred)
    maete = mean_absolute_error(y_test, y_predte)

    r2t = r2_score(y_train, y_predt)
    r2 = r2_score(y_val, y_pred)
    r2te = r2_score(y_test, y_predte)

    rmset = np.sqrt(mset)
    rmse = np.sqrt(mse)
    rmsete = np.sqrt(msete)

    # Create a new DataFrame to store the metrics for the current model
    metrics_df = pd.DataFrame({
        "Metric": ["MSE", "MAE", "R²", "RMSE"],
        "Train": [mset, maet, r2t, rmset],
        "Validation": [mse, mae, r2, rmse],
        "Test": [msete, maete, r2te, rmsete]
    })

    # Store the DataFrame in the dictionary with the model name as the key
    model_metrics_tables[model_name] = metrics_df

# Evaluate all models
for result in best_models:
    model = result["model"]
    model_name = type(model.named_steps['regressor']).__name__ if 'regressor' in model.named_steps else type(model).__name__

    # Evaluate the model and store the results
    evaluate_model(model, X_train, y_train, X_val, y_val, X_test, y_test, model_name)

# Print each model's metrics table separately
for model_name, metrics_df in model_metrics_tables.items():
    print(f"Metrics for {model_name}:")
    print(tabulate(metrics_df, headers='keys', tablefmt='grid'))
    print("\n")


Metrics for LinearRegression:
+----+----------+---------------+---------------+---------------+
|    | Metric   |         Train |    Validation |          Test |
+====+==========+===============+===============+===============+
|  0 | MSE      | 107684        | 140927        | 108779        |
+----+----------+---------------+---------------+---------------+
|  1 | MAE      |     71.8811   |     81.324    |     71.2697   |
+----+----------+---------------+---------------+---------------+
|  2 | R²       |      0.879097 |      0.907768 |      0.900455 |
+----+----------+---------------+---------------+---------------+
|  3 | RMSE     |    328.152    |    375.403    |    329.816    |
+----+----------+---------------+---------------+---------------+


Metrics for Ridge:
+----+----------+---------------+--------------+---------------+
|    | Metric   |         Train |   Validation |          Test |
+====+==========+===============+==============+===============+
|  0 | MSE      | 107684    